In [1]:
import tensorflow as tf
import random

In [2]:
mnist = tf.keras.datasets.mnist
( x_train, y_train ), ( x_test, y_test ) = mnist.load_data()

# Nomalization
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = tf.cast( x_train, tf.float32 )
x_test = tf.cast( x_test, tf.float32 )

print( x_train.shape, y_train.shape )

(60000, 28, 28) (60000,)


In [3]:
nb_classes = 10

In [4]:
# Reshape
x_train = tf.reshape( x_train, [ -1, 784 ] )
x_test = tf.reshape( x_test, [ -1, 784 ] )
print( x_train.shape, x_test.shape )

(60000, 784) (10000, 784)


In [5]:
# one-hot Encoding
y_train = tf.keras.utils.to_categorical( y_train, nb_classes )
y_test = tf.keras.utils.to_categorical( y_test, nb_classes )

print( y_train.shape, y_test.shape )

(60000, 10) (10000, 10)


In [6]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [7]:
xavier = tf.keras.initializers.GlorotUniform()
W1 = tf.Variable( xavier( [ 784, 512 ] ) )
b1 = tf.Variable( tf.random.normal( [ 512 ] ) )
def Layer1( X ):
    return ( tf.nn.relu( tf.matmul( X, W1 ) + b1 ) )

W2 = tf.Variable( xavier( [ 512, 512 ] ) )
b2 = tf.Variable( tf.random.normal( [ 512 ] ) )
def Layer2( X ):
    return ( tf.nn.relu( tf.matmul( Layer1( X ), W2 ) + b2 ) )

W3 = tf.Variable( xavier( [ 512, 512 ] ) )
b3 = tf.Variable( tf.random.normal( [ 512 ] ) )
def Layer3( X ):
    return ( tf.nn.relu( tf.matmul( Layer2( X ), W3 ) + b3 ) )

W4 = tf.Variable( xavier( [ 512, 512 ] ) )
b4 = tf.Variable( tf.random.normal( [ 512 ] ) )
def Layer4( X ):
    return ( tf.nn.relu( tf.matmul( Layer3( X ), W4 ) + b4 ) )

W5 = tf.Variable( xavier( [ 512, nb_classes ] ) )
b5 = tf.Variable( tf.random.normal( [ nb_classes ] ) )

@tf.function
def Hypothesis( X ):
    return (  tf.matmul( Layer4( X ), W5 ) + b5 )

In [8]:
@tf.function
def Cost( X, Y ):
    return ( tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits = Hypothesis( X ), labels = Y )
    ) )

def Minimize( X, Y ):
    loss = lambda: Cost( X ,Y )
    
    tf.keras.optimizers.Adam( learning_rate ).minimize( loss, [ W1, W2, W3, W4, W5,
                                                               b1, b2, b3, b4, b5 ] )

In [9]:
def CorrectPrediction( X, Y ):
    return ( tf.equal( tf.argmax( Hypothesis( X ), axis = 1 ),
                      tf.argmax( Y, axis = 1 ) ) )

def Accuracy( X, Y ):
    return ( tf.reduce_mean( tf.cast( CorrectPrediction( X, Y ), tf.float32 ) ) )

In [10]:
for epoch in range( training_epochs ):
    avg_cost = 0
    total_batch = int( len( x_train ) / batch_size )
    
    start_batch, end_batch = 0, batch_size
    for i in range( total_batch ):
        batch_xs, batch_ys = \
            x_train[ start_batch : end_batch ], y_train[start_batch : end_batch ]
        
        Minimize( batch_xs, batch_ys )
        cost_val = Cost( batch_xs, batch_ys )
        
        avg_cost += cost_val / total_batch
        
        start_batch = start_batch + batch_size
        end_batch = end_batch + batch_size
    
    print( 'Epoch: {:04d}, Cost: {:.9f}'.format( epoch + 1, avg_cost ) )
    
print( 'Learning finished' )
        
    

Epoch: 0001, Cost: 0.399486929
Epoch: 0002, Cost: 0.243531823
Epoch: 0003, Cost: 0.106234506
Epoch: 0004, Cost: 0.071614519
Epoch: 0005, Cost: 0.047530562
Epoch: 0006, Cost: 0.038594950
Epoch: 0007, Cost: 0.030863831
Epoch: 0008, Cost: 0.028324831
Epoch: 0009, Cost: 0.024764160
Epoch: 0010, Cost: 0.015634175
Epoch: 0011, Cost: 0.016665097
Epoch: 0012, Cost: 0.013412203
Epoch: 0013, Cost: 0.011950971
Epoch: 0014, Cost: 0.010452059
Epoch: 0015, Cost: 0.011448454
Learning finished


In [11]:
tf.print( 'Accuracy: ', Accuracy( x_test, y_test ))

Accuracy:  0.9722


In [12]:
r = random.randint( 0, len( x_test ) - 1 )
tf.print( 'Label: ', tf.argmax( y_test[ r: r + 1 ], axis = 1 ) )
tf.print( 
    'Prediction: ', tf.argmax(Hypothesis( x_test[ r: r + 1] ), axis = 1 ) )

Label:  [0]
Prediction:  [0]
